# Strojenie hiperparametrów

## Zadania

### Poszukiwanie ręczne

Pobierz zestaw danych Boston Housing:

In [105]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

Przygotuj funkcję budującą model według parametrów podanych jako argumenty:
- n_hidden – liczba warstw ukrytych,
- n_neurons – liczba neuronów na każdej z warstw ukrytych,
- optimizer – gradientowy algorytm optymalizacji, funkcja powinna rozumieć wartości: sgd, nesterov, momentum oraz adam,
- learning_rate – krok uczenia,
- momentum – współczynnik przyspieszenia dla algorytmów z pędem.

In [106]:
X_train

array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

In [107]:
def build_model(optimizer, n_hidden=1, learning_rate=10e-5, n_neurons=25, momentum=0):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train.shape[1:]))
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    if optimizer == "momentum":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer, metrics=['mae'])
    return model

Przy uczeniu wykorzystaj mechanizm early stopping o cierpliwości równej 10 i minimalnej poprawie
funkcji straty równej 1.00, uczenie maksymalnie przez 100 epok.

In [108]:
es = tf.keras.callbacks.EarlyStopping(patience=10,
                                      min_delta=1.00)

In [109]:
def get_run_logdir(name, value):
    import time
    import os
    root_logdir = os.path.join(os.curdir, "tb_logs")
    ts = int(time.time())

    run_id = str(ts) + "_" + name + "_" + str(value)
    return os.path.join(root_logdir, run_id)




Przed eksperymentami wyczyść sesję TensorFlow i ustal generatory liczb losowych:

In [110]:
import numpy as np

tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [111]:
epochs = 100
validation_split = .1

krok uczenia(lr): 10−6, 10−5, 10−4

In [112]:
results_lr = []
for lr in (10e-6, 10e-5, 10e-4):
    run_logdir = get_run_logdir("lr", lr)
    tensorboard = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(learning_rate=lr, optimizer="sgd")
    model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, callbacks=[es, tensorboard])
    score = model.evaluate(X_test, y_test)
    results_lr.append((lr, score[0], score[1]))

Epoch 1/100
12/12 [==============================] - 1s 14ms/step - loss: 117101.7812 - mae: 123.7935 - val_loss: 108.1399 - val_mae: 8.5337
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 163.8253 - mae: 9.5988 - val_loss: 68.6999 - val_mae: 6.7683
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 113.2871 - mae: 7.8936 - val_loss: 49.6265 - val_mae: 5.5180
Epoch 4/100
12/12 [==============================] - 0s 5ms/step - loss: 97.5250 - mae: 7.2386 - val_loss: 44.5197 - val_mae: 5.2958
Epoch 5/100
12/12 [==============================] - 0s 5ms/step - loss: 91.5391 - mae: 7.0076 - val_loss: 42.7563 - val_mae: 5.2981
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 87.4378 - mae: 7.0010 - val_loss: 42.5495 - val_mae: 5.0830
Epoch 7/100
12/12 [==============================] - 0s 5ms/step - loss: 86.9827 - mae: 6.7954 - val_loss: 41.1674 - val_mae: 5.1446
Epoch 8/100
12/12 [==============================] - 0s 5ms

In [113]:
results_lr

[(1e-05, 96.47820281982422, 8.2672700881958),
 (0.0001, 423.1081848144531, 18.43541145324707),
 (0.001, 658428.0, 811.384521484375)]

liczba warstw ukrytych (hl): od 0 do 3,

In [114]:
results_hl = []
for hl in (0, 1, 2, 3):
    run_logdir = get_run_logdir("hl", hl)
    tensorboard = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(n_hidden=hl, optimizer="sgd")
    model.summary()
    model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, callbacks=[es, tensorboard])
    score = model.evaluate(X_test, y_test)
    results_hl.append((hl, score[0], score[1]))


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1)                 14        
                                                                 
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
12/12 [==============================] - 1s 18ms/step - loss: inf - mae: 462933766606063403008.0000 - val_loss: inf - val_mae: 967319372936436208631808.0000
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
12/12 [==============================] - 0s 4ms/

In [115]:
results_hl

[(0, nan, nan),
 (1, 432.7367858886719, 18.69472885131836),
 (2, 83.7286148071289, 6.532285213470459),
 (3, nan, nan)]

liczba neuronów na warstwę (nn): 5, 25, 125

In [116]:
results_nn = []
for nn in (5, 25, 125):
    run_logdir = get_run_logdir("nn", nn)
    tensorboard = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(optimizer="sgd", n_neurons=nn)
    model.summary()
    model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, callbacks=[es, tensorboard])
    score = model.evaluate(X_test, y_test)
    results_nn.append((nn, score[0], score[1]))


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 5)                 70        
                                                                 
 dense_17 (Dense)            (None, 1)                 6         
                                                                 
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
12/12 [==============================] - 0s 14ms/step - loss: 14374.3311 - mae: 54.7305 - val_loss: 500.3336 - val_mae: 21.3929
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 588.5240 - mae: 22.3446 - val_loss: 498.0657 - val_mae: 21.3398
Epoch 3/100
12/12 [==============================] - 0s 5ms/step - loss: 586.1573 - mae: 22.2912 - val_loss: 495.8081 - val_mae: 21.2868
Epoch 4/100
12/12 [=====================

In [117]:
results_nn

[(5, 415.259033203125, 18.221284866333008),
 (25, 828.7384643554688, 27.303747177124023),
 (125, 438.08441162109375, 18.83722686767578)]

algorytm optymalizacji (opt): wszystkie 4 algorytmy (pęd = 0.5)

In [118]:
results_alg = []
for alg in ("sgd", "nesterov", "momentum", "adam"):
    run_logdir = get_run_logdir("alg", alg)
    tensorboard = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(optimizer=alg, momentum=.5)
    model.summary()
    model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, callbacks=[es, tensorboard])
    score = model.evaluate(X_test, y_test)
    results_alg.append((alg, score[0], score[1]))


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 25)                350       
                                                                 
 dense_23 (Dense)            (None, 1)                 26        
                                                                 
Total params: 376
Trainable params: 376
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
12/12 [==============================] - 0s 13ms/step - loss: 427640096.0000 - mae: 6068.5132 - val_loss: 50.2310 - val_mae: 5.8455
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 68.1092 - mae: 5.6119 - val_loss: 44.7272 - val_mae: 5.4800
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 84.9172 - mae: 6.5169 - val_loss: 63.2436 - val_mae: 6.5680
Epoch 4/100
12/12 [======================

In [119]:
results_alg

[('sgd', 62.78519821166992, 5.925008773803711),
 ('nesterov', 340.63629150390625, 16.072277069091797),
 ('momentum', 310.46917724609375, 15.184233665466309),
 ('adam', 144.28643798828125, 9.175629615783691)]

 pęd (mom): 0.1, 0.5, 0.9 (dla algorytmu momentum).

In [120]:
results_mom = []
for mom in (.1, .5, .9):
    run_logdir = get_run_logdir("momentum", mom)
    tensorboard = tf.keras.callbacks.TensorBoard(run_logdir)
    model = build_model(optimizer="momentum", momentum=mom)
    model.summary()
    model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, callbacks=[es, tensorboard])
    score = model.evaluate(X_test, y_test)
    results_mom.append((mom, score[0], score[1]))


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 25)                350       
                                                                 
 dense_31 (Dense)            (None, 1)                 26        
                                                                 
Total params: 376
Trainable params: 376
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
12/12 [==============================] - 0s 13ms/step - loss: 18974574.0000 - mae: 1321.2195 - val_loss: 649.2841 - val_mae: 24.6293
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 743.3138 - mae: 25.5749 - val_loss: 645.9587 - val_mae: 24.5617
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 739.8649 - mae: 25.5069 - val_loss: 642.6512 - val_mae: 24.4943
Epoch 4/100
12/12 [=============

In [121]:
results_mom

[(0.1, 493.1888427734375, 20.24709701538086),
 (0.5, 254.59796142578125, 13.4472017288208),
 (0.9, 171.73822021484375, 11.43071174621582)]

### Automatyczne poszukiwanie przestrzeni argumentów

Przygotuj słownik zawierający przeszukiwane wartości parametrów

In [138]:
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": [5, 25, 125],
    "learning_rate": [10e-6, 10e-5, 10e-4],
    "optimizer": ["sgd", "adam", "nesterov", "momentum"],
    "momentum": [.1, .5, .9]
}

Przygotuj callback early stopping i obuduj przygotowaną wcześniej funkcję build_model obiektem
KerasRegressor

In [140]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

C:\Users\fudal\AppData\Local\Temp/ipykernel_17604/461105605.py:4: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = KerasRegressor(build_model, callbacks=[es])


In [147]:
from sklearn.model_selection import RandomizedSearchCV

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=30, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Epoch 1/100
8/8 [==============================] - 1s 24ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 7ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Ep

C:\ProgramData\Anaconda3\envs\untitled\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [            nan -5.18409805e+01             nan -6.67791176e+01
 -8.01099080e+03 -2.56524972e+02 -5.72521642e+01 -5.51805585e+01
 -6.09553363e+01 -4.73256772e+04 -6.04231962e+01 -1.36446545e+06
 -6.71445612e+04 -6.76066775e+25 -2.46250505e+02 -9.42294617e+01
 -4.17691355e+02 -7.20970245e+02 -3.00464732e+02 -2.46198772e+02
 -8.62007243e+01 -7.61032842e+01             nan             nan
             nan -9.70506420e+13 -2.81771966e+04             nan
             nan -1.38826374e+03]
  warnings.warn(


12/12 [==============================] - 0s 13ms/step - loss: 3550.7024 - mae: 57.3489 - val_loss: 1080.1235 - val_mae: 31.8331
Epoch 2/100
12/12 [==============================] - 0s 4ms/step - loss: 572.4300 - mae: 20.2508 - val_loss: 111.1474 - val_mae: 8.9311
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 193.0645 - mae: 10.8997 - val_loss: 163.4747 - val_mae: 10.3957
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 182.0846 - mae: 11.1496 - val_loss: 70.5926 - val_mae: 6.9998
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 104.3986 - mae: 7.6499 - val_loss: 40.3460 - val_mae: 4.9520
Epoch 6/100
12/12 [==============================] - 0s 4ms/step - loss: 95.2306 - mae: 6.6114 - val_loss: 38.0113 - val_mae: 4.9032
Epoch 7/100
12/12 [==============================] - 0s 4ms/step - loss: 89.0306 - mae: 6.5991 - val_loss: 37.4515 - val_mae: 4.9935
Epoch 8/100
12/12 [==============================] - 0s 4ms/step

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000245E213ABF0>,
                   n_iter=30,
                   param_distributions={'learning_rate': [1e-05, 0.0001, 0.001],
                                        'momentum': [0.1, 0.5, 0.9],
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': [5, 25, 125],
                                        'optimizer': ['sgd', 'adam', 'nesterov',
                                                      'momentum']},
                   verbose=2)

Zapisz najlepsze znalezione parametry w postaci słownika do pliku rnd_search.pkl.

In [148]:
rnd_search_cv.best_params_

{'optimizer': 'adam',
 'n_neurons': 125,
 'n_hidden': 2,
 'momentum': 0.5,
 'learning_rate': 0.0001}

dict

In [149]:
import pickle

with open('lr.pkl', 'wb') as file:
    pickle.dump(results_lr, file)

with open('hl.pkl', 'wb') as file:
    pickle.dump(results_hl, file)

with open('nn.pkl', 'wb') as file:
    pickle.dump(results_nn, file)

with open('opt.pkl', 'wb') as file:
    pickle.dump(results_alg, file)

with open('mom.pkl', 'wb') as file:
    pickle.dump(results_mom, file)

with open('rnd_search.pkl', 'wb') as file:
    pickle.dump(rnd_search_cv.best_params_, file)

[(0.1, 493.1888427734375, 20.24709701538086),
 (0.5, 254.59796142578125, 13.4472017288208),
 (0.9, 171.73822021484375, 11.43071174621582)]